# Introduction & Business Problem

### Introduction
As a business and enterpreneurship enthusiast, i'd like to look for a business opportunities and gives my thought on:
what the people are currently need? is it highly available on the market? if it isnt, what can i do to meet those demands? how should I do it?


### Business Problem
The problem in answering above question, it would take quite sometime to gather information needed in a conventional way. That is why, we are going to need an efficient way to do it, which called data science.

Using data science, we could tackle the problems:
1. Using data to determine which business opportunity that is/are available
2. Spending money on investing required detailed and thorough analysis, which take quite some time to be done conventional way

#### Who would be interested in this project?
1. Investor looking for an opportunity to build a business
2. People that was looking for a preferred neighborhood to live


By knowing what people needs, we can use it to our advantage, to make profit from it.
To start it all, first i need to know what are the frequently visited venue category in Toronto?
Because to my understanding, the more a venue category is available, the higher demands it has received from the market.
not only that, i also need information on who visited those venues? is the population of toronto can be grouped by ethnics?
Is what these ethnics needs already widely available in Toronto?



------------------------------------

# Data

Now that i have defined the question, what data do i need to have in order to answer said question?

a. Foursquare Location Data

b. Ethnics demographic population in Toronto

These are the data I needed, where the informations i sought were stored.
Population & Demographics data of Toronto from Wikipedia will be used to:
1. Maps the majority of ethnics that lived in Toronto
2. How many (headcount & percentage) people that belongs in that ethnics?
3. Where the majority of ethnics are located (borough & neighborhood)?

the Foursquare Location Data will be used to:
4. maps the available venue category available in Toronto
5. which venue category are amongst the top frequently visited by people of Toronto


having knowing the 1st, 2nd, and 5th data, we could determine what venue category that still has room to grow in Toronto for business opportunity, what business would potentilly thrive and where should we build it.

Meanwhile from the 3rd and 4th data, we could predict which neighborhood that potentially be the second "home" for the majority ethnics? by clustering the neighborhood based on its similarity in venue categories.

------------------------------

## Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

#importing beautifulsoup
from bs4 import BeautifulSoup

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.
Folium installed
Libraries imported.


## Importing Dataset from Wikipedia

### Borough & Neighborhood datasets

In [2]:
#sending request
wiki_table= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html = wiki_table.text
html


'\n<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoFTzgpAICoAAD@fNTsAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":948084252,"wgRevisionId":948084252,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related l

In [3]:
# Extract Table from the source
soup = BeautifulSoup(html,'lxml')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoFTzgpAICoAAD@fNTsAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":948084252,"wgRevisionId":948084252,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [4]:
#Exploring where the data might be classified
#my_table = soup.find('table',{'class':'wikitable sortable'})
my_table = soup.find('table',{'class':'wikitable'})
my_table

<table class="wikitable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East York
<

In [8]:
#Getting Column Header and Data Rows
tableheaders = [th.text.strip('\n') for th in my_table.find_all('th')]
tablerows = [[td.text.strip('\n') for td in row.find_all('td')] for row in my_table.find_all('tr')[1:]]

#transform the column header and data row into pandas dataframe
df = pd.DataFrame(tablerows,columns = tableheaders)
df.head()


#replace column names
df.rename(columns={'Postal code': 'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Preprocessing

In [9]:
#replace 'Not Assigned' rows with 'NaN' for easier processing

df.replace('Not assigned', np.nan, inplace = True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,
1,M2A,NaN,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [10]:
#filter-out rows with 'NaN' values in column 'Borough'

df_c = df[df['Borough'].notna()]
df_c.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


#### Group the data by Borough

In [11]:
df_g = df_c.groupby(['PostalCode','Borough'])['Neighborhood'] \
                    .apply(', '.join) \
                    .reset_index(name='Neighborhood')

print(df_g.shape)
df_g.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Download Coordinates Data

In [12]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2020-04-01 02:39:29--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228, 159.8.69.21, 159.8.69.24
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-01 02:39:31--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197, 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-04-01 02:39:31--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5

In [13]:
#convert downloaded data into a pandas dataframe

coor_data = pd.read_csv('Geospatial_Coordinates.csv', delimiter=",")
coor_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
#Rename 'Postal Code' column header in Coordinates Data into 'PostalCode' to match with Neighborhood Data

coor_data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
coor_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Combine Neighborhood Data with Coordinates Data

In [15]:
#Merge Neighborhood data with Coordinates Data by using 'PostalCode' as reference

df_complete = pd.merge(df_g, coor_data[['PostalCode','Latitude', 'Longitude']], on='PostalCode')
df_complete.head()

#test
df_complete[df_complete['Neighborhood']=='Islington Avenue']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


----

### Demographics & Population Data

In [16]:
#sending request
wiki_table2= requests.get('https://en.wikipedia.org/wiki/Demographics_of_Toronto')
html2 = wiki_table2.text

soup2 = BeautifulSoup(html2,'lxml')
print(soup.prettify())

#Exploring where the data might be classified
my_table2 = soup2.find('table',{'class':'wikitable sortable'})
my_table2

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoFTzgpAICoAAD@fNTsAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":948084252,"wgRevisionId":948084252,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

<table class="wikitable sortable">
<tbody><tr>
<th colspan="8">Visible minority and Aboriginal population<sup class="reference" id="cite_ref-30"><a href="#cite_note-30">[30]</a></sup><sup class="reference" id="cite_ref-31"><a href="#cite_note-31">[31]</a></sup><sup class="reference" id="cite_ref-32"><a href="#cite_note-32">[32]</a></sup>
</th></tr>
<tr>
<th colspan="2">Population group</th>
<th>Population (<a href="/wiki/2016_Canadian_Census" title="2016 Canadian Census">2016</a>)</th>
<th>% of total population (2016)</th>
<th>Population (<a href="/wiki/2011_Canadian_Census" title="2011 Canadian Census">2011</a>)</th>
<th>% of total population (2011)</th>
<th>Population (<a href="/wiki/2006_Canadian_Census" title="2006 Canadian Census">2006</a>)</th>
<th>% of total population (2006)
</th></tr>
<tr>
<td colspan="2"><b><a href="/wiki/European_Canadians" title="European Canadians">European (White)</a></b></td>
<td><b>1,282,750</b></td>
<td><b><span data-sort-value="7001476563762578181♠" s

Since this wikipedia page contains multiple table, let's select the table that we need by locating the last sentence prior the table, and the first sentence after the table

as you can see from the wikipedia page, the last sentence right before the table is:
"The most prevalent ethnic origins in the City of Toronto are as follows:"

and the first sentence right after the table is:
"The top visible-minority groups per Community Council (2016 Census)"


to get only the data table that we need, we have to locate the index of those sentence and then select the data in between the specified index.

In [17]:
#find the index of the last sentence right before the table and the first sentence right after the table

print(html2.index('The most prevalent ethnic origins in the City of Toronto are as follows'))
print(html2.index('The top visible-minority groups per'))

57955
63359


In [22]:
#select the desired data using the substring from previously defined index
html_table = html2[html2.index('The most prevalent ethnic origins in the City of Toronto are as follows'):html2.index('The top visible-minority groups per')]

# Extract Table from the source
soup2 = BeautifulSoup(html_table,'lxml')
print(soup2.prettify())


#Select the data using its class
my_table2 = soup2.find('table',{'class':'wikitable sortable'})



<html>
 <body>
  <p>
   The most prevalent ethnic origins in the City of Toronto are as follows:
  </p>
  <table class="wikitable sortable">
   <tbody>
    <tr>
     <th>
      Top 20
      <a href="/wiki/Ethnic_origins_of_people_in_Canada" title="Ethnic origins of people in Canada">
       Ethnic origins
      </a>
      in the City of Toronto
     </th>
     <th>
      Population (2016)
      <sup class="reference" id="cite_ref-StatCan2011EO_33-0">
       <a href="#cite_note-StatCan2011EO-33">
        [33]
       </a>
      </sup>
     </th>
     <th>
      Percentage
     </th>
     <th>
      <i>
       Riding with Highest Concentration
      </i>
     </th>
     <th>
      <i>
       Percentage
      </i>
     </th>
    </tr>
    <tr>
     <td>
      <a href="/wiki/Chinese_Canadians" title="Chinese Canadians">
       Chinese
      </a>
     </td>
     <td>
      332,830
     </td>
     <td>
      12.5
     </td>
     <td>
      <i>
       <a href="/wiki/Scarborough%E2%80%94Agincou

In [23]:
#Getting Column Header and Data Rows

tablerows2 = [[td.text.strip('\n') for td in row.find_all('td')] for row in my_table2.find_all('tr')[1:]]
tableheaders2 = [th.text.strip('\n') for th in my_table2.find_all('th')]

print(tableheaders2)
print(tablerows2)

['Top 20 Ethnic origins in the City of Toronto', 'Population (2016)[33]', 'Percentage', 'Riding with Highest Concentration', 'Percentage']
[['Chinese', '332,830', '12.5', 'Scarborough—Agincourt', '47.0'], ['English', '331,890', '12.3', 'Beaches—East York', '24.2'], ['Canadian', '323,175', '12.0', 'Beaches-East York', '19.7'], ['Irish', '262,965', '9.8', 'Parkdale—High Park', '20.0'], ['Scottish', '256,250', '9.5', 'Beaches—East York', '18.9'], ['East Indian', '202,675', '7.5', 'Etobicoke North', '22.2'], ['Italian', '182,495', '6.8', 'Etobicoke Centre', '15.1'], ['Filipino', '162,605', '6.0', 'York Centre', '17.0'], ['German', '130,900', '4.9', 'Parkdale—High Park', '9.8'], ['French', '122,615', '4.6', 'Parkdale—High Park', '8.9'], ['Polish', '114,530', '4.3', 'Eglinton—Lawrence', '12.0'], ['Portuguese', '100,420', '3.7', 'Davenport', '22.7'], ['Jamaican', '90,065', '3.3', 'Humber River—Black Creek', '8.5'], ['Russian', '74,465', '2.8', 'York Centre', '9.5'], ['Ukrainian', '72,340', '2

Since the header has 2 similar column (percentage), let's make new columns to clear it up

In [24]:
new_header = ['Ethnics', 'Population (2016)', 'Population Percentage in Toronto', 'Riding with Highest Concentration', 'Percentage in Riding']
new_header

['Ethnics',
 'Population (2016)',
 'Population Percentage in Toronto',
 'Riding with Highest Concentration',
 'Percentage in Riding']

In [30]:
#Create Dataframe

dff = pd.DataFrame(tablerows2,columns = new_header)
dff.head(100)

,Ethnics,Population (2016),Population Percentage in Toronto,Riding with Highest Concentration,Percentage in Riding
0,Chinese,"332,830",12.5,Scarborough—Agincourt,47.0
1,English,"331,890",12.3,Beaches—East York,24.2
2,Canadian,"323,175",12.0,Beaches-East York,19.7
3,Irish,"262,965",9.8,Parkdale—High Park,20.0
4,Scottish,"256,250",9.5,Beaches—East York,18.9
5,East Indian,"202,675",7.5,Etobicoke North,22.2
6,Italian,"182,495",6.8,Etobicoke Centre,15.1
7,Filipino,"162,605",6.0,York Centre,17.0
8,German,"130,900",4.9,Parkdale—High Park,9.8
9,French,"122,615",4.6,Parkdale—High Park,8.9


In [31]:
#split column "Riding with Highest Concentration" into Borough & Neighborhood

new = dff['Riding with Highest Concentration'].str.split('—', n = 1, expand = True)

dff['Borough']= new[0]
dff['Neighborhood'] = new[1]
dff.drop(columns = ['Riding with Highest Concentration'], inplace = True)

dff

,Ethnics,Population (2016),Population Percentage in Toronto,Percentage in Riding,Borough,Neighborhood
0,Chinese,"332,830",12.5,47.0,Scarborough,Agincourt
1,English,"331,890",12.3,24.2,Beaches,East York
2,Canadian,"323,175",12.0,19.7,Beaches-East York,None
3,Irish,"262,965",9.8,20.0,Parkdale,High Park
4,Scottish,"256,250",9.5,18.9,Beaches,East York
5,East Indian,"202,675",7.5,22.2,Etobicoke North,None
6,Italian,"182,495",6.8,15.1,Etobicoke Centre,None
7,Filipino,"162,605",6.0,17.0,York Centre,None
8,German,"130,900",4.9,9.8,Parkdale,High Park
9,French,"122,615",4.6,8.9,Parkdale,High Park


In [32]:
#drop the last row (total population)

dff.drop([20], axis = 0, inplace = True)
dff.tail()

,Ethnics,Population (2016),Population Percentage in Toronto,Percentage in Riding,Borough,Neighborhood
15,Sri Lankan,"58,180",2.2,11.1,Scarborough,Rouge Park
16,Greek,"57,425",2.1,7.3,Toronto,Danforth
17,Spanish,"56,815",2.1,< 5.0,,None
18,British Isles origins (other),"52,900",2.0,< 5.0,,None
19,Iranian,"45,540",1.7,12.1,Willowdale,None


In [33]:
dff.head()

,Ethnics,Population (2016),Population Percentage in Toronto,Percentage in Riding,Borough,Neighborhood
0,Chinese,"332,830",12.5,47.0,Scarborough,Agincourt
1,English,"331,890",12.3,24.2,Beaches,East York
2,Canadian,"323,175",12.0,19.7,Beaches-East York,None
3,Irish,"262,965",9.8,20.0,Parkdale,High Park
4,Scottish,"256,250",9.5,18.9,Beaches,East York


2016 data shows that the Chinese is the most prevalent ethnics in Toronto, with a total of 332,830 headcounts, which about 12.5% of the entire Toronto population that year. About 47% of them lived in Agincourt, Scarborough.

Now that we already have the Population & Demographic Data, let's continue for Location Data.

------------

--------------

## Set Up Foursquare Location Data

In [34]:
CLIENT_ID = 'SN3X4BETS0SNEV32LT20Y5AUMFOC0LPONRA4R0BOJ5G3H14H' # your Foursquare ID
CLIENT_SECRET = 'P43IPKXGNRKB2QDEHUZJNLIMSATC5AHIEMEIZELK5KW505CJ' # your Foursquare Secret
VERSION = '20200226' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SN3X4BETS0SNEV32LT20Y5AUMFOC0LPONRA4R0BOJ5G3H14H
CLIENT_SECRET:P43IPKXGNRKB2QDEHUZJNLIMSATC5AHIEMEIZELK5KW505CJ


In [35]:
LIMIT = 100
radius = 500


## Combining Location data from Foursquare with Neighborhood Data 


In [36]:
#Function to get nearby venues for each borough

def getNearbyVenues(borough, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, name, lat, lng in zip(borough, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
#Getting Nearby venues
toronto_venues = getNearbyVenues(borough=df_complete['Borough'],
                                 names=df_complete['Neighborhood'],
                                   latitudes=df_complete['Latitude'],
                                   longitudes=df_complete['Longitude']
                                 )


Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

In [39]:
toronto_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


# Methodology 1

### Determining what are the most frequently visited venue category

In [40]:
# How many unique venue categories?

print('There are {} uniques venues categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques venues categories.


In [41]:
toronto_venues[['Venue Category', 'Borough']].groupby(['Venue Category'])['Borough'] \
                                             .count().reset_index(name='count') \
                                             .sort_values(['count'], ascending = False)


,Venue Category,count
60,Coffee Shop,188
48,Café,97
217,Restaurant,73
203,Pizza Place,53
198,Park,49
154,Italian Restaurant,48
19,Bakery,43
21,Bar,41
155,Japanese Restaurant,41
145,Hotel,38


#### Coffee Shop & Cafe occupied both first and second place as the most visited venue in Toronto. As you can see, Restaurant still has room grow.

#### Let's see what kind of restaurant category that can be found across Toronto.

In [42]:
toronto_restaurant=toronto_venues[toronto_venues['Venue Category'].str.contains('Restaurant')]

toronto_restaurant.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
5,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
14,Scarborough,Woburn,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
15,Scarborough,Cedarbrae,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
16,Scarborough,Cedarbrae,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant


In [43]:
toronto_restaurant[['Venue Category', 'Borough']].groupby(['Venue Category'])['Borough'] \
                                                 .count().reset_index(name='count') \
                                                 .sort_values(['count'], ascending = False)

,Venue Category,count
40,Restaurant,73
27,Italian Restaurant,48
28,Japanese Restaurant,41
1,American Restaurant,30
18,Fast Food Restaurant,26
42,Sushi Restaurant,26
41,Seafood Restaurant,25
44,Thai Restaurant,25
7,Chinese Restaurant,20
47,Vegetarian / Vegan Restaurant,18


In [44]:
#display 10 restaurant category with the restaurant count in Toronto

toronto_restaurant[['Venue Category', 'Borough']].groupby(['Venue Category'])['Borough'] \
                                                 .count().reset_index(name='count') \
                                                 .sort_values(['count'], ascending = False).head(10)

,Venue Category,count
40,Restaurant,73
27,Italian Restaurant,48
28,Japanese Restaurant,41
1,American Restaurant,30
18,Fast Food Restaurant,26
42,Sushi Restaurant,26
41,Seafood Restaurant,25
44,Thai Restaurant,25
7,Chinese Restaurant,20
47,Vegetarian / Vegan Restaurant,18


From the list above, the most common restaurant that can be found is "Restaurant", followed by Italian and Japanese Restaurant. 
What's surprising, Chinese restaurant is at the 9th position of the most common restaurant category here in Toronto, while Chinese Ethnics counted for 12.5% of the whole Toronto Population.

This findings shows that there are a lot of room to grow for Chinese Restaurant in Toronto, for 332,830 people of Chinese Ethnics there are only 22 Chinese Restaurant available in the whole Toronto Area.


Let's see where the Chinese Restaurant are all located


In [45]:
#Filter only Chinese Restaurant in Venue Category column

tvr= toronto_venues[toronto_venues['Venue Category'].str.contains('Chinese')]
tvr

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
28,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,Hakka No.1,43.727688,-79.266057,Chinese Restaurant
44,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,Kim Kim restaurant,43.753833,-79.276611,Chinese Restaurant
60,Scarborough,Clarks Corners / Tam O'Shanter / Sullivan,43.781638,-79.304302,The Royal Chinese Restaurant 避風塘小炒,43.780505,-79.298844,Chinese Restaurant
75,Scarborough,Steeles West / L'Amoreaux West,43.799525,-79.318389,Mr Congee Chinese Cuisine 龍粥記,43.798879,-79.318335,Chinese Restaurant
76,Scarborough,Steeles West / L'Amoreaux West,43.799525,-79.318389,Phoenix Restaurant 金鳳餐廳,43.798198,-79.318432,Chinese Restaurant
150,North York,Fairview / Henry Farm / Oriole,43.778517,-79.346556,Manchu Wok,43.778225,-79.343302,Chinese Restaurant
160,North York,Bayview Village,43.786947,-79.385975,Sun Star Chinese Cuisine 翠景小炒,43.787914,-79.381234,Chinese Restaurant
236,North York,Don Mills,43.725900,-79.340923,Congee Star 帝王名粥,43.726586,-79.341833,Chinese Restaurant
257,North York,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,China Court,43.755780,-79.437437,Chinese Restaurant
481,Central Toronto,North Toronto West,43.715383,-79.405678,C'est Bon,43.716785,-79.400406,Chinese Restaurant


Drop Latitude & Longitude Column

In [46]:
#Select available column
tvr.columns[:]



Index(['Borough', 'Neighborhood', 'Neighborhood Latitude',
       'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude',
       'Venue Category'],
      dtype='object')

In [47]:
#Remove Longitude and Latitude column

tvr_c = tvr.drop(toronto_venues.columns[[2, 3, 5, 6]], axis = 1) 
tvr_c

,Borough,Neighborhood,Venue,Venue Category
28,Scarborough,Kennedy Park / Ionview / East Birchmount Park,Hakka No.1,Chinese Restaurant
44,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,Kim Kim restaurant,Chinese Restaurant
60,Scarborough,Clarks Corners / Tam O'Shanter / Sullivan,The Royal Chinese Restaurant 避風塘小炒,Chinese Restaurant
75,Scarborough,Steeles West / L'Amoreaux West,Mr Congee Chinese Cuisine 龍粥記,Chinese Restaurant
76,Scarborough,Steeles West / L'Amoreaux West,Phoenix Restaurant 金鳳餐廳,Chinese Restaurant
150,North York,Fairview / Henry Farm / Oriole,Manchu Wok,Chinese Restaurant
160,North York,Bayview Village,Sun Star Chinese Cuisine 翠景小炒,Chinese Restaurant
236,North York,Don Mills,Congee Star 帝王名粥,Chinese Restaurant
257,North York,Bathurst Manor / Wilson Heights / Downsview North,China Court,Chinese Restaurant
481,Central Toronto,North Toronto West,C'est Bon,Chinese Restaurant


# Result 1

Count how many Chinese Restaurant is / are available in each Borough & Neighborhood

In [48]:
#Group by each Borough 
tvr_c.groupby('Borough').count()

,Neighborhood,Venue,Venue Category
Borough,,,
Central Toronto,1,1,1
Downtown Toronto,9,9,9
Etobicoke,1,1,1
North York,4,4,4
Scarborough,5,5,5


only 4 Chinese Restaurant are available in Scarborough, despite roughly 47% of Chinese people in Toronto are all live there. 

Now let's see in what neighborhood the Chinese Restaurant are all Located

In [49]:
#Chinese Restaurant Location Grouped by Borough & Neighborhood 
tvr_c.groupby(['Borough', 'Neighborhood']).count()

Venue  \
Borough          Neighborhood                                                
Central Toronto  North Toronto West                                      1   
Downtown Toronto Central Bay Street                                      1   
                 Garden District, Ryerson                                1   
                 Harbourfront East / Union Station / Toronto Isl...      1   
                 Kensington Market / Chinatown / Grange Park             2   
                 St. James Town / Cabbagetown                            2   
                 Toronto Dominion Centre / Design Exchange               1   
                 University of Toronto / Harbord                         1   
Etobicoke        Westmount                                               1   
North York       Bathurst Manor / Wilson Heights / Downsview North       1   
                 Bayview Village                                         1   
                 Don Mills                                               1   
                 Fairview / Henry Farm / Oriole                          1   
Scarborough      Clarks Corners / Tam O'Shanter / Sullivan               1   
                 Dorset Park / Wexford Heights / Scarborough Tow...      1   
                 Kennedy Park / Ionview / East Birchmount Park           1   
                 Steeles West / L'Amoreaux West                          2   

                                                                     Venue Category  
Borough          Neighborhood                                                        
Central Toronto  North Toronto West                                               1  
Downtown Toronto Central Bay Street                                               1  
                 Garden District, Ryerson                                         1  
                 Harbourfront East / Union Station / Toronto Isl...               1  
                 Kensington Market / Chinatown / Grange Park                      2  
                 St. James Town / Cabbagetown                                     2  
                 Toronto Dominion Centre / Design Exchange                        1  
                 University of Toronto / Harbord                                  1  
Etobicoke        Westmount                                                        1  
North York       Bathurst Manor / Wilson Heights / Downsview North                1  
                 Bayview Village                                                  1  
                 Don Mills                                                        1  
                 Fairview / Henry Farm / Oriole                                   1  
Scarborough      Clarks Corners / Tam O'Shanter / Sullivan                        1  
                 Dorset Park / Wexford Heights / Scarborough Tow...               1  
                 Kennedy Park / Ionview / East Birchmount Park                    1  
                 Steeles West / L'Amoreaux West                                   2

None of the Chinese Restaurant are available in Agincourt! home to roughly 156,430 chinese people in Toronto. This means they have to travel to other neighborhoods in Scarborough, or even further to Downtown Toronto to get any chinese dishes.

# Discussion 1

With this revelation, now we know what kind of restaurant that still has a lot of room to grow, which is a Chinese Restaurant based on the fact that Chinese Ethnic has the most population in Toronto compared to any other Ethnics (12.5% of Toronto, or about 332,830 people), where majority of the Chinese are lived in Agincourt - Scarborough. This findings later strongly supported with the fact that there are lack of Chinese Restaurant availability, only 22 Chinese Restaurant can be found in whole Toronto, pale in comparison against Italian Restaurant. Not to mention, there are only 4 Chinese Restaurants available in Scarborough, and none of them are located in Agincourt.

Thus, if I want to build a restaurant in Toronto, based on the data that i have, I would build a Chinese Restaurant in Agincourt, home of 47% (156,430 headcounts) chinese people in Toronto.

---

# Next Opportunity

Potential area to build my next restaurant

If I want to set up another Chinese Restaurant in Toronto, i would pick a neighborhood where it will likely to be the second Agincourt. 
1. Why refers to Agincourt? because as the above data suggest majority of Chinese People lived in Agincourt, that means there are some things that made Agincourt are prefered for them to live
2. How? I can predict it by clustering other neighborhood that shares the same similarity with Agincourt. What makes Agincourt is a desired place to be called a home by Chinese Ethnics could be determined by the availability and variety of Venue Category there. Therefore, I should find similar neighborhoods by clustering the neighborhoods based on the most common venue category for each neighborhood. 

# Methodology 2

## Pre Processing

In [50]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
toronto_onehot['Borough'] = toronto_venues['Borough']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2197, 271)


,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhood']).mean().reset_index()
toronto_grouped.head()

,Borough,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,Davisville,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

#### Print each neighborhood with its 5 top most common venue

In [52]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1        Dessert Shop  0.09
2      Sandwich Place  0.09
3  Italian Restaurant  0.06
4    Sushi Restaurant  0.06


----Davisville North----
              venue  freq
0    Breakfast Spot  0.14
1               Gym  0.14
2    Sandwich Place  0.14
3  Department Store  0.14
4             Hotel  0.14


----Forest Hill North & West----
                venue  freq
0    Sushi Restaurant  0.25
1               Trail  0.25
2       Jewelry Store  0.25
3  Mexican Restaurant  0.25
4   Accessories Store  0.00


----Lawrence Park----
                       venue  freq
0         Photography Studio  0.25
1                       Park  0.25
2                Swim School  0.25
3                   Bus Line  0.25
4  Middle Eastern Restaurant  0.00


----Moore Park / Summerhill East----
                             venue  freq
0                     Tennis Court   1.0
1                Accessories Store   0.0
2        Middle Eastern Restaur

                  venue  freq
0     Convenience Store  0.07
1         Grocery Store  0.07
2         Burrito Place  0.07
3           Social Club  0.07
4  Fast Food Restaurant  0.07


----New Toronto / Mimico South / Humber Bay Shores----
                  venue  freq
0           Pizza Place  0.09
1   American Restaurant  0.09
2  Fast Food Restaurant  0.09
3          Liquor Store  0.09
4   Fried Chicken Joint  0.09


----Northwest----
                 venue  freq
0  Rental Car Location   0.5
1            Drugstore   0.5
2    Accessories Store   0.0
3   Miscellaneous Shop   0.0
4  Monument / Landmark   0.0


----Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East----
                        venue  freq
0  Construction & Landscaping  0.50
1              Baseball Field  0.25
2               Deli / Bodega  0.25
3           Accessories Store  0.00
4           Mobile Phone Shop  0.00


----South Steeles /

                       venue  freq
0                     Bakery  0.25
1             Breakfast Spot  0.25
2                 Smoke Shop  0.25
3  Middle Eastern Restaurant  0.25
4         Miscellaneous Shop  0.00


----Woburn----
                 venue  freq
0          Coffee Shop  0.67
1    Korean Restaurant  0.33
2   Miscellaneous Shop  0.00
3                Motel  0.00
4  Monument / Landmark  0.00


----Brockton / Parkdale Village / Exhibition Place----
                venue  freq
0                Café  0.13
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3  Italian Restaurant  0.04
4   Convenience Store  0.04


----Dufferin / Dovercourt Village----
           venue  freq
0         Bakery  0.11
1       Pharmacy  0.11
2           Bank  0.06
3  Grocery Store  0.06
4    Supermarket  0.06


----High Park / The Junction South----
                 venue  freq
0                  Bar  0.08
1   Mexican Restaurant  0.08
2      Thai Restaurant  0.08
3                 Café  0.08
4  Arts & 

#### Convert to Pandas, and sort the venues in descending order


In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
# Create new dataframe and display top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']


for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Thai Restaurant,Seafood Restaurant,Central Toronto
1,Davisville North,Gym,Park,Breakfast Spot,Department Store,Hotel,Sandwich Place,Food & Drink Shop,Donut Shop,Distribution Center,Dog Run,Central Toronto
2,Forest Hill North & West,Jewelry Store,Sushi Restaurant,Trail,Mexican Restaurant,Diner,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Central Toronto
3,Lawrence Park,Park,Photography Studio,Swim School,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Central Toronto
4,Moore Park / Summerhill East,Tennis Court,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Fast Food Restaurant,Central Toronto


----------------

## Clustering the Neighborhoods

To be able to cluster several datapoints (in this case, neighborhood) based on its feature, one of the Machine Learning that I can use is K-Means Clustering. By defining how many cluster that we need, we can determine which neighborhood that has similarity with each other.

### run k-means to cluster the neighborhood into 20 clusters.

In [55]:
num_clusters = 20

toronto_grouped_clustering = toronto_grouped.drop(['Neighborhood', 'Borough'], axis= 1)

k_means = KMeans(init='k-means++', n_clusters= num_clusters, n_init=12).fit(toronto_grouped_clustering)
labels = k_means.labels_

print(labels)


[ 3  3  3  3  7  3  9  3  3  3  3  3  1  3  3  3  3  3  3  3  3  3 13  3
  3  3  3  1  3  3  3  3  3  8  3  0  3  3  0  3 17  3  0 12 10  0 19  0
  3  3  1  3  3  3  3  3  3 11  5  3 10  3  8  3  3 15  8  3  1  0  0 14
  3  3  3  3  2 13 16  4  0  1  6  3  1  1  3  3  3 13  3 18 17]


In [56]:
toronto_grouped_clustering.shape

(93, 269)

In [57]:
#add labels column into the df
toronto_grouped_clustering["Labels"] = labels
toronto_grouped_clustering.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Labels
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

### merge cluster labels into Location & Neighborhood dataset

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', labels)

toronto_merged = df_complete

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index(['Borough', 'Neighborhood']), on=['Borough', 'Neighborhood'])

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,16.0,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3.0,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
3,M1G,Scarborough,Woburn,43.770992,-79.216917,6.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant


New dataframe that includes cluster labels and top 10 venues for each neighborhood

### Cleansing

In [60]:
#Look for null values in dataset

toronto_merged[pd.isnull(toronto_merged).any(axis=1)]
toronto_merged['Cluster Labels'].unique()

array([ 2., 16.,  3.,  6.,  0.,  4., 14.,  1., 13., nan, 15.,  8.,  7.,
        9., 18., 10., 17., 19., 11.,  5., 12.])

#### make sure no Null values available and the 'Cluster Labels' values are formatted in integer

In [61]:
#Filter out null values

toronto_merged_c = toronto_merged[toronto_merged['Cluster Labels'].notna()]
toronto_merged_c['Cluster Labels'].unique()

array([ 2., 16.,  3.,  6.,  0.,  4., 14.,  1., 13., 15.,  8.,  7.,  9.,
       18., 10., 17., 19., 11.,  5., 12.])

In [62]:
# change data format of column 'Cluster Labels' from float to integer

toronto_merged_c['Cluster Labels'] = toronto_merged_c['Cluster Labels'].astype(pd.np.int64)
toronto_merged_c.head(20)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,16,Bar,History Museum,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
3,M1G,Scarborough,Woburn,43.770992,-79.216917,6,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Caribbean Restaurant,Bakery,Thai Restaurant,Fried Chicken Joint,Athletics & Sports,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Grocery Store,Playground,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,3,Department Store,Convenience Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,14,American Restaurant,Motel,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,1,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


Now that we clustered the Neighborhood based on its similarity in top 10 Most Common Venues, let's see what cluster does Agincourt belongs to

In [63]:
toronto_merged_c[toronto_merged_c['Neighborhood']=='Agincourt']

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1S,Scarborough,Agincourt,43.7942,-79.262029,3,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store


As you can see, Agincourt included in cluster Label number 3. Now, What are the other Neighborhoods that fall into the same Cluster Label as Agincourt but still within the same Borough (Agincourt belongs in Scarborough) ?

In [64]:
toronto_merged_cs = toronto_merged_c[toronto_merged_c['Cluster Labels']==3]
print(toronto_merged_cs[toronto_merged_cs['Borough']=='Scarborough'].shape)

toronto_merged_css = toronto_merged_cs[toronto_merged_cs['Borough']=='Scarborough']
toronto_merged_css


(5, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,3,Department Store,Convenience Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant
10,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,3,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store


There are 4 other Neighborhoods that are similar with Agincourt in terms of top 10 Most Common Venue. To narrow the options, let's see how many Venue Category available in each Neighborhood?

Let's start with excluding neighborhoods that were already has a Chinese Restaurant

In [65]:
#Chinese Restaurant Location Grouped by Borough & Neighborhood 
tvr_c[tvr_c['Borough']=='Scarborough'].groupby(['Borough', 'Neighborhood']).count()

Venue  \
Borough     Neighborhood                                                
Scarborough Clarks Corners / Tam O'Shanter / Sullivan               1   
            Dorset Park / Wexford Heights / Scarborough Tow...      1   
            Kennedy Park / Ionview / East Birchmount Park           1   
            Steeles West / L'Amoreaux West                          2   

                                                                Venue Category  
Borough     Neighborhood                                                        
Scarborough Clarks Corners / Tam O'Shanter / Sullivan                        1  
            Dorset Park / Wexford Heights / Scarborough Tow...               1  
            Kennedy Park / Ionview / East Birchmount Park                    1  
            Steeles West / L'Amoreaux West                                   2

In [67]:
#Drop neighborhood that belongs in the same cluster as agincourt, (cluster label no 3) that already has a chinese restaurant: Dorset Park / Wexford Heights / Scarborough Town Centre, Kennedy Park / Ion View / East Birchmount Park eighborhoods 

toronto_merged_cssd = toronto_merged_css.drop([6, 10], axis = 0)
toronto_merged_cssd


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store


Down to 3 other similars neighborhood. Let's see the total availability of venue category in each neighborhood

In [75]:
toronto_venuess= toronto_venues[toronto_venues['Borough']=='Scarborough'].groupby(['Borough', 'Neighborhood']).count().reset_index()
toronto_venuess.drop(toronto_venuess.columns[2:7], axis = 1, inplace= True)
toronto_venuess.drop([1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15], inplace= True)

toronto_venuess

,Borough,Neighborhood,Venue Category
0,Scarborough,Agincourt,5
6,Scarborough,Golden Mile / Clairlea / Oakridge,8
7,Scarborough,Guildwood / Morningside / West Hill,9


Since I Am going to build a Chinese Restaurant, let's narrow it down to how many other restaurants that would be my competitor are available in each neighborhood

In [83]:
#toronto_restaurant

toronto_restaurants= toronto_restaurant[toronto_restaurant['Borough']=='Scarborough'].groupby(['Borough', 'Neighborhood']).count().reset_index()
toronto_restaurants.drop(toronto_restaurant.columns[2:7], axis = 1, inplace= True)
toronto_restaurants.drop([1, 3, 4, 6, 8, 9], inplace= True)

toronto_restaurants

,Borough,Neighborhood,Venue Category
0,Scarborough,Agincourt,1
2,Scarborough,Clarks Corners / Tam O'Shanter / Sullivan,4
5,Scarborough,Guildwood / Morningside / West Hill,1
7,Scarborough,Malvern / Rouge,1
10,Scarborough,Woburn,1


Golden Mile / Clairlea / Oakridge Neighborhoods dont have any restaurant available

# Results 2

### Cluster the neighborhood and display them on the map!

In [91]:
#Getting Geographical Coordinate of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Cluster 1: neighborhoods within Scarborough that have restaurant available, but share the same similarity with Agincourt

In [84]:
#drop Agincourt, Dorset Park / Wexford Heights / Scarborough Town Centre, 

cluster1 = toronto_merged_css.drop([7], axis = 0)
cluster1


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,3,Department Store,Convenience Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
10,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,3,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store


In [92]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster1['Latitude'], cluster1['Longitude'], cluster1['Neighborhood'], cluster1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

--

### Cluster 2: neighborhoods within Scarborough that have Chinese Restaurant Available, but share the same similarity with Agincourt

In [89]:
cluster2 = toronto_merged_css.drop([2, 7], axis = 0)
cluster2


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,3,Department Store,Convenience Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
10,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.757410,-79.273304,3,Indian Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
12,M1S,Scarborough,Agincourt,43.794200,-79.262029,3,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Discount Store


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster2['Latitude'], cluster2['Longitude'], cluster2['Neighborhood'], cluster2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

--

### Cluster 3: neighborhoods within Scarborough that dont have any Chinese Restaurant Available, but share the same similarity with Agincourt

In [95]:
cluster3 = toronto_merged_css.drop([6, 10, 12], axis = 0)
cluster3


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,3,Moving Target,Spa,Breakfast Spot,Rental Car Location,Electronics Store,Bank,Intersection,Medical Center,Mexican Restaurant,Yoga Studio
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster3['Latitude'], cluster3['Longitude'], cluster3['Neighborhood'], cluster3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

--

### Cluster 4: neighborhoods that dont have any restaurant available at all

In [96]:
#neighborhoods that dont have any restaurant available

cluster4 = toronto_merged_css.drop([2, 6, 10, 12], axis = 0)
cluster4

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Bakery,Park,Ice Cream Shop,Bus Line,Metro Station,Intersection,Soccer Field,Coworking Space,Creperie,Falafel Restaurant


In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cluster4['Latitude'], cluster4['Longitude'], cluster4['Neighborhood'], cluster4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

------

# Discussion 2

With above data, now i know where should i build my next Chinese Restaurant outside of Agincourt, but still within Scarborough where no Chinese Restaurant are currently available. It would be:
1. Guildwood, Morningside, West Hill


Also with above data, I also know where should i build my next Chinese Restaurant outside of Agincourt, but still within Scarborough where no restaurant are currently available at all, it would be:
1. Golden Mile / Clairlea / Oakridge



------------------------------

# Conclusions

1. If i want to build a new restaurant business in Toronto, that would be a Chinese Restaurant in Agincourt, Scarborough. Why? because Chinese people are the most prevalent ethnics in Toronto (12.5% of Toronto population or 332,830 people), where most of them lived in Scarborough - Agincourt (47 % of total Chinese population or 156,430 people) while not even one chinese restaurant can be found in Agincourt.


2. For potential area to build my next Chinese Restaurant, I have potential places that I can pick that i grouped based on:
    
    2.1. Outside of Agincourt, but still within Scarborough where no restaurant are currently available, either:
        2.1.1. Golden Mile / Clairlea / Oakridge
    
    2.2. Outside of Agincourt, but still within Scarborough where there are several non-Chinese restaurant already established. It would be either:
        2.2.1. Guildwood, Morningside, West Hill



# Thankyou!